In [6]:
# ================================
# Notebook 4: Modeling & Thresholds
# ================================

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Any

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, roc_curve,
    precision_score, recall_score, f1_score, confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import optuna
import json

# ---------------- Config ----------------
DATA_DIR = Path("artifacts_preprocessing")
TARGET = "loan_status"
RANDOM_STATE = 42
N_TRIALS = 100  # increase if you want deeper search

# Optional: minimum precision floor when picking thresholds (None = no floor)
MIN_PREC_FOR_RECALL = None

# -------------- Load Splits --------------
train_df = pd.read_csv(DATA_DIR / "train.csv")
val_df   = pd.read_csv(DATA_DIR / "val.csv")
test_df  = pd.read_csv(DATA_DIR / "test.csv")

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET].astype(int)

X_val = val_df.drop(columns=[TARGET])
y_val = val_df[TARGET].astype(int)

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET].astype(int)

print("Shapes | Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# Basic imbalance info (for class weights / scale_pos_weight)
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = neg / max(1, pos)
print(f"Train positives: {pos}, negatives: {neg}, scale_pos_weight≈{scale_pos_weight:.2f}")

# ---------- Helpers: metrics & thresholds ----------
def evaluate_probs(y_true, y_prob, thr=0.5) -> Dict[str, Any]:
    y_pred = (y_prob >= thr).astype(int)
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "cm": confusion_matrix(y_true, y_pred).tolist()
    }

def pick_thresholds_from_roc(y_true, y_prob):
    # ROC-based thresholds
    fpr, tpr, thr = roc_curve(y_true, y_prob)
    thr = np.nan_to_num(thr, nan=0.5)

    # (A) Youden's J
    j_scores = tpr - fpr
    idx_j = int(np.argmax(j_scores))
    thr_j = float(thr[idx_j])

    # (B) Closest to (0,1)
    dist = np.sqrt((1 - tpr)**2 + (fpr)**2)
    idx_c = int(np.argmin(dist))
    thr_c = float(thr[idx_c])

    # (C) Optional: maximize recall subject to precision floor on VAL (if set)
    thr_best_recall = None
    if MIN_PREC_FOR_RECALL is not None:
        # Sweep a simple grid
        cand = np.unique(np.concatenate([thr, np.linspace(0, 1, 201)]))
        best_r = -1
        best_t = 0.5
        for t in cand:
            m = evaluate_probs(y_true, y_prob, t)
            if m["precision"] >= MIN_PREC_FOR_RECALL and m["recall"] > best_r:
                best_r, best_t = m["recall"], t
        thr_best_recall = float(best_t)

    return {
        "youden_j": thr_j,
        "closest_01": thr_c,
        "recall_with_precision_floor": thr_best_recall
    }

def summarize(model_name, split_name, y_true, y_prob, thr_dict):
    rows = []
    for label, t in thr_dict.items():
        if t is None: 
            continue
        m = evaluate_probs(y_true, y_prob, t)
        rows.append({
            "model": model_name,
            "split": split_name,
            "threshold_method": label,
            "threshold": round(t, 4),
            **{k: (round(v,4) if isinstance(v, float) else v) for k, v in m.items() if k != "cm"},
            "cm": m["cm"]
        })
    # also add default 0.5
    m05 = evaluate_probs(y_true, y_prob, 0.5)
    rows.append({
        "model": model_name, "split": split_name, "threshold_method": "fixed_0.5",
        "threshold": 0.5, **{k:(round(v,4) if isinstance(v,float) else v) for k,v in m05.items() if k!="cm"},
        "cm": m05["cm"]
    })
    return rows

# ---------- 1) Baseline Logistic Regression ----------
# class_weight='balanced' to counter imbalance
lr = LogisticRegression(max_iter=200, class_weight="balanced", solver="liblinear", random_state=RANDOM_STATE)
lr.fit(X_train, y_train)

val_prob_lr = lr.predict_proba(X_val)[:,1]
thr_lr = pick_thresholds_from_roc(y_val, val_prob_lr)

# choose Youden J as the main ROC-based threshold
t_lr = thr_lr["youden_j"]
test_prob_lr = lr.predict_proba(X_test)[:,1]

report_rows = []
report_rows += summarize("Logistic (baseline)", "val",  y_val,  val_prob_lr,  thr_lr)
report_rows += summarize("Logistic (baseline)", "test", y_test, test_prob_lr, thr_lr)

print("Logistic baseline done.")

# ---------- 2) RandomForest: baseline ----------
# class_weight from compute_class_weight
cw_vals = compute_class_weight("balanced", classes=np.array([0,1]), y=y_train)
cw = {0: cw_vals[0], 1: cw_vals[1]}

rf_base = RandomForestClassifier(
    n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1, class_weight=cw
)
rf_base.fit(X_train, y_train)

val_prob_rf_base = rf_base.predict_proba(X_val)[:,1]
thr_rf_base = pick_thresholds_from_roc(y_val, val_prob_rf_base)
report_rows += summarize("RandomForest (baseline)", "val",  y_val,  val_prob_rf_base, thr_rf_base)
report_rows += summarize("RandomForest (baseline)", "test", y_test, rf_base.predict_proba(X_test)[:,1], thr_rf_base)

print("RF baseline done.")

# ---------- 3) RandomForest: Optuna tuning ----------
def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 800),
        "max_depth": trial.suggest_int("max_depth", 4, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
    }
    mdl = RandomForestClassifier(
        random_state=RANDOM_STATE, n_jobs=-1, class_weight=cw, **params
    )
    mdl.fit(X_train, y_train)
    prob = mdl.predict_proba(X_val)[:,1]
    return average_precision_score(y_val, prob)  # optimize PR-AUC; ROC-AUC is also fine

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=N_TRIALS, show_progress_bar=False)

rf_tuned = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1, class_weight=cw, **study_rf.best_params)
rf_tuned.fit(X_train, y_train)

val_prob_rf_tuned = rf_tuned.predict_proba(X_val)[:,1]
thr_rf_tuned = pick_thresholds_from_roc(y_val, val_prob_rf_tuned)
report_rows += summarize("RandomForest (tuned)", "val",  y_val,  val_prob_rf_tuned, thr_rf_tuned)
report_rows += summarize("RandomForest (tuned)", "test", y_test, rf_tuned.predict_proba(X_test)[:,1], thr_rf_tuned)

print("RF tuned done. Best params:", study_rf.best_params)

# ---------- 4) XGBoost: baseline ----------
xgb_base = XGBClassifier(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
    eval_metric="logloss", tree_method="hist", n_jobs=-1,
    random_state=RANDOM_STATE, scale_pos_weight=scale_pos_weight
)
xgb_base.fit(X_train, y_train)
val_prob_xgb_base = xgb_base.predict_proba(X_val)[:,1]
thr_xgb_base = pick_thresholds_from_roc(y_val, val_prob_xgb_base)
report_rows += summarize("XGBoost (baseline)", "val",  y_val,  val_prob_xgb_base, thr_xgb_base)
report_rows += summarize("XGBoost (baseline)", "test", y_test, xgb_base.predict_proba(X_test)[:,1], thr_xgb_base)

print("XGB baseline done.")

# ---------- 5) XGBoost: Optuna tuning ----------
def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 10.0, log=True),
    }
    mdl = XGBClassifier(
        eval_metric="logloss", tree_method="hist", n_jobs=-1,
        random_state=RANDOM_STATE, scale_pos_weight=scale_pos_weight, **params
    )
    mdl.fit(X_train, y_train)
    prob = mdl.predict_proba(X_val)[:,1]
    return average_precision_score(y_val, prob)

study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_xgb, n_trials=N_TRIALS, show_progress_bar=False)

xgb_tuned = XGBClassifier(
    eval_metric="logloss", tree_method="hist", n_jobs=-1,
    random_state=RANDOM_STATE, scale_pos_weight=scale_pos_weight, **study_xgb.best_params
)
xgb_tuned.fit(X_train, y_train)

val_prob_xgb_tuned = xgb_tuned.predict_proba(X_val)[:,1]
thr_xgb_tuned = pick_thresholds_from_roc(y_val, val_prob_xgb_tuned)
report_rows += summarize("XGBoost (tuned)", "val",  y_val,  val_prob_xgb_tuned, thr_xgb_tuned)
report_rows += summarize("XGBoost (tuned)", "test", y_test, xgb_tuned.predict_proba(X_test)[:,1], thr_xgb_tuned)

print("XGB tuned done. Best params:", study_xgb.best_params)

# ---------- 6) LightGBM: baseline ----------
lgb_base = LGBMClassifier(
    n_estimators=500, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8,
    random_state=RANDOM_STATE, n_jobs=-1,
    class_weight={0:1.0, 1: scale_pos_weight}  # simple weighting
)
lgb_base.fit(X_train, y_train)
val_prob_lgb_base = lgb_base.predict_proba(X_val)[:,1]
thr_lgb_base = pick_thresholds_from_roc(y_val, val_prob_lgb_base)
report_rows += summarize("LightGBM (baseline)", "val",  y_val,  val_prob_lgb_base, thr_lgb_base)
report_rows += summarize("LightGBM (baseline)", "test", y_test, lgb_base.predict_proba(X_test)[:,1], thr_lgb_base)

print("LGBM baseline done.")

# ---------- 7) LightGBM: Optuna tuning ----------
def objective_lgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 15, 255),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-3, 10.0, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    mdl = LGBMClassifier(
        random_state=RANDOM_STATE, n_jobs=-1,
        class_weight={0:1.0, 1: scale_pos_weight}, **params
    )
    mdl.fit(X_train, y_train)
    prob = mdl.predict_proba(X_val)[:,1]
    return average_precision_score(y_val, prob)

study_lgb = optuna.create_study(direction="maximize")
study_lgb.optimize(objective_lgb, n_trials=N_TRIALS, show_progress_bar=False)

lgb_tuned = LGBMClassifier(
    random_state=RANDOM_STATE, n_jobs=-1,
    class_weight={0:1.0, 1: scale_pos_weight}, **study_lgb.best_params
)
lgb_tuned.fit(X_train, y_train)

val_prob_lgb_tuned = lgb_tuned.predict_proba(X_val)[:,1]
thr_lgb_tuned = pick_thresholds_from_roc(y_val, val_prob_lgb_tuned)
report_rows += summarize("LightGBM (tuned)", "val",  y_val,  val_prob_lgb_tuned, thr_lgb_tuned)
report_rows += summarize("LightGBM (tuned)", "test", y_test, lgb_tuned.predict_proba(X_test)[:,1], thr_lgb_tuned)

print("LGBM tuned done. Best params:", study_lgb.best_params)

# ---------- 8) Consolidated Report ----------
report_df = pd.DataFrame(report_rows)

# Nice ordering
cols = ["model","split","threshold_method","threshold",
        "roc_auc","pr_auc","precision","recall","f1","cm"]
report_df = report_df[cols].sort_values(["split","model","threshold_method"]).reset_index(drop=True)

# Save
OUT_DIR = Path("artifacts_modeling")
OUT_DIR.mkdir(parents=True, exist_ok=True)
report_df.to_csv(OUT_DIR / "consolidated_report.csv", index=False)

# Also save best params to a json
best_params = {
    "rf_tuned": study_rf.best_params,
    "xgb_tuned": study_xgb.best_params,
    "lgb_tuned": study_lgb.best_params
}
with open(OUT_DIR / "best_params.json", "w") as f:
    json.dump(best_params, f, indent=2)

print("\n✅ Done. Consolidated results:")
display(report_df.head(20))
print(f"\nSaved: {OUT_DIR / 'consolidated_report.csv'}")
print(f"Best params saved: {OUT_DIR / 'best_params.json'}")

# ---------- 9) Quick summary printout (how performance improved) ----------
def quick_summary(df, split="test"):
    d = df[df["split"]==split]
    # pick each model's Youden-J row as the main comparator
    main = d[d["threshold_method"]=="youden_j"].copy()
    # Add Logistic baseline fixed_0.5 too for reference (if you prefer)
    return main.sort_values("roc_auc", ascending=False)

print("\n=== Progress on TEST (Using Youden-J thresholds) ===")
display(quick_summary(report_df, "test"))


Shapes | Train: (19548, 26) Val: (6516, 26) Test: (6517, 26)
Train positives: 4265, negatives: 15283, scale_pos_weight≈3.58
Logistic baseline done.


[I 2025-10-31 22:59:17,598] A new study created in memory with name: no-name-bbc06cfe-0a7d-4ae5-91ad-25fa43f4cc28


RF baseline done.


[I 2025-10-31 22:59:18,189] Trial 0 finished with value: 0.8673327296138021 and parameters: {'n_estimators': 218, 'max_depth': 25, 'min_samples_split': 10, 'min_samples_leaf': 19, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8673327296138021.
[I 2025-10-31 22:59:18,941] Trial 1 finished with value: 0.8205841789372655 and parameters: {'n_estimators': 436, 'max_depth': 5, 'min_samples_split': 18, 'min_samples_leaf': 11, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8673327296138021.
[I 2025-10-31 22:59:25,338] Trial 2 finished with value: 0.8837895471022724 and parameters: {'n_estimators': 615, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 2 with value: 0.8837895471022724.
[I 2025-10-31 22:59:33,357] Trial 3 finished with value: 0.8802186712937945 and parameters: {'n_estimators': 756, 'max_depth': 21, 'min_samples_split': 18, 'min_samples_leaf': 11, 'max_features': None}. Best is trial 2 with value: 0.8837895471022

RF tuned done. Best params: {'n_estimators': 557, 'max_depth': 28, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


...

[I 2025-10-31 23:24:39,302] Trial 99 finished with value: 0.9030888841018514 and parameters: {'n_estimators': 814, 'learning_rate': 0.02555546786993943, 'num_leaves': 60, 'feature_fraction': 0.6060257369973997, 'bagging_fraction': 0.8103412750460997, 'lambda_l2': 0.00225788249993722, 'min_child_samples': 50}. Best is trial 31 with value: 0.9040500905973347.


[LightGBM] [Warning] feature_fraction is set=0.6060257369973997, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6060257369973997
[LightGBM] [Warning] bagging_fraction is set=0.8103412750460997, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8103412750460997
[LightGBM] [Warning] lambda_l2 is set=0.00225788249993722, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00225788249993722
[LightGBM] [Warning] feature_fraction is set=0.6362302663851724, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6362302663851724
[LightGBM] [Warning] bagging_fraction is set=0.949452100017126, subsample=1.0 will be ignored. Current value: bagging_fraction=0.949452100017126
[LightGBM] [Warning] lambda_l2 is set=0.0024291070542918312, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0024291070542918312
[LightGBM] [Warning] feature_fraction is set=0.6362302663851724, colsample_bytree=1.0 will be ignored. Current value: featur

,model,split,threshold_method,threshold,roc_auc,pr_auc,precision,recall,f1,cm
0,LightGBM (baseline),test,closest_01,0.3201,0.9486,0.9072,0.6937,0.8586,0.7674,"[[4556, 539], [201, 1221]]"
1,LightGBM (baseline),test,fixed_0.5,0.5000,0.9486,0.9072,0.8786,0.7890,0.8314,"[[4940, 155], [300, 1122]]"
2,LightGBM (baseline),test,youden_j,0.4213,0.9486,0.9072,0.8044,0.8186,0.8114,"[[4812, 283], [258, 1164]]"
3,LightGBM (tuned),test,closest_01,0.3227,0.9475,0.9051,0.7275,0.8432,0.7811,"[[4646, 449], [223, 1199]]"
4,LightGBM (tuned),test,fixed_0.5,0.5000,0.9475,0.9051,0.8664,0.7890,0.8259,"[[4922, 173], [300, 1122]]"
5,LightGBM (tuned),test,youden_j,0.4103,0.9475,0.9051,0.8094,0.8150,0.8122,"[[4822, 273], [263, 1159]]"
6,Logistic (baseline),test,closest_01,0.5532,0.8261,0.6237,0.4727,0.7314,0.5743,"[[3935, 1160], [382, 1040]]"
7,Logistic (baseline),test,fixed_0.5,0.5000,0.8261,0.6237,0.4298,0.7799,0.5542,"[[3624, 1471], [313, 1109]]"
8,Logistic (baseline),test,youden_j,0.5532,0.8261,0.6237,0.4727,0.7314,0.5743,"[[3935, 1160], [382, 1040]]"
9,RandomForest (baseline),test,closest_01,0.1867,0.9282,0.8802,0.6588,0.8242,0.7323,"[[4488, 607], [250, 1172]]"



Saved: artifacts_modeling/consolidated_report.csv
Best params saved: artifacts_modeling/best_params.json

=== Progress on TEST (Using Youden-J thresholds) ===


,model,split,threshold_method,threshold,roc_auc,pr_auc,precision,recall,f1,cm
2,LightGBM (baseline),test,youden_j,0.4213,0.9486,0.9072,0.8044,0.8186,0.8114,"[[4812, 283], [258, 1164]]"
17,XGBoost (baseline),test,youden_j,0.4567,0.9484,0.9064,0.8330,0.8066,0.8196,"[[4865, 230], [275, 1147]]"
20,XGBoost (tuned),test,youden_j,0.4170,0.9477,0.9057,0.8533,0.7975,0.8244,"[[4900, 195], [288, 1134]]"
5,LightGBM (tuned),test,youden_j,0.4103,0.9475,0.9051,0.8094,0.8150,0.8122,"[[4822, 273], [263, 1159]]"
14,RandomForest (tuned),test,youden_j,0.2801,0.9332,0.8854,0.8409,0.7658,0.8016,"[[4889, 206], [333, 1089]]"
11,RandomForest (baseline),test,youden_j,0.2200,0.9282,0.8802,0.7300,0.8024,0.7645,"[[4673, 422], [281, 1141]]"
8,Logistic (baseline),test,youden_j,0.5532,0.8261,0.6237,0.4727,0.7314,0.5743,"[[3935, 1160], [382, 1040]]"
